In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import joblib
import os

# التأكد من وجود المجلدات
os.makedirs('../models', exist_ok=True)
os.makedirs('../data', exist_ok=True)
os.makedirs('../results/evaluation', exist_ok=True)

# تحميل البيانات
data = pd.read_csv('../data/selected_features.csv')  # تصحيح اسم الملف
X = data

# تحميل عمود target من البيانات الأصلية
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data"
columns = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target']
data_original = pd.read_csv(url, names=columns)
data_original = data_original.replace('?', np.nan)
data_original['ca'] = data_original['ca'].astype(float)
data_original['thal'] = data_original['thal'].astype(float)
data_original = data_original.fillna(data_original.mean())
y = (data_original['target'] > 0).astype(int)  # تحويل إلى تصنيف ثنائي

# تقسيم البيانات إلى تدريب واختبار
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# تحديد شبكة المعلمات لـ Random Forest
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# إنشاء النموذج
rf = RandomForestClassifier(random_state=42)

# تهيئة المعلمات باستخدام GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

# عرض أفضل المعلمات والنتيجة
print(f"أفضل المعلمات: {grid_search.best_params_}")
print(f"أفضل دقة على مجموعة التدريب (CV): {grid_search.best_score_:.4f}")

# تقييم النموذج النهائવضل النموذج النهائي على مجموعة الاختبار
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
results = {
    'Model': 'Tuned Random Forest',
    'Accuracy': accuracy_score(y_test, y_pred),
    'Precision': precision_score(y_test, y_pred, average='weighted', zero_division=0),
    'Recall': recall_score(y_test, y_pred, average='weighted', zero_division=0),
    'F1-Score': f1_score(y_test, y_pred, average='weighted', zero_division=0)
}

# حفظ النتائج
results_df = pd.DataFrame([results])
results_df.to_csv('../results/evaluation/tuned_model_metrics.csv', index=False)
print("\nنتائج النموذج النهائي على مجموعة الاختبار:")
print(results_df)

# حفظ النموذج النهائي
joblib.dump(best_model, '../models/final_model.pkl')

# حفظ أفضل المعلمات
best_params_df = pd.DataFrame([grid_search.best_params_])
best_params_df.to_csv('../results/evaluation/best_parameters.csv', index=False)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits
أفضل المعلمات: {'bootstrap': True, 'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 100}
أفضل دقة على مجموعة التدريب (CV): 0.8180

نتائج النموذج النهائي على مجموعة الاختبار:
                 Model  Accuracy  Precision    Recall  F1-Score
0  Tuned Random Forest  0.901639   0.903684  0.901639  0.901692
